In [163]:
%matplotlib inline

import pandas as pd
import numpy as np
from scipy.stats import rv_continuous, norm, truncnorm, arcsine, chi2, gamma, triang, entropy
import matplotlib.pyplot as plt
from itertools import combinations_with_replacement
from time import time
from random import shuffle
from copy import deepcopy

In [2]:
gift_types = ['horse', 'ball', 'bike', 'train', 'coal', 'book', 'doll', 'blocks', 'gloves']
gift_numbers = [1000, 1100, 500, 1000, 166, 1200, 1000, 1000, 200]
gift_ids = [[item +'_'+str(i) for i in range(gift_numbers[index])] for index, item in enumerate(gift_types)]
df = pd.read_csv('best_combinations.csv')

In [446]:
def unshared_copy(inList):
    if isinstance(inList, list):
        return list( map(unshared_copy, inList) )
    return inList

def multiple_pop(li, npop):
    out = []
    for i in range(npop):
        out += [li.pop()]
    return out

def recipe_list_to_ids(recipe, gift_ids):
    return ' '.join([' '.join(multiple_pop(gift_ids[i], n)) for i,n in enumerate(recipe) if n > 0])

def add_to_cart(cart, recipe, items_left):
    recipe = list(map(int, recipe))
    items_left = [i-r if (i-r)>=0 else exec('raise Exception(\'Negative number of items left.\')') for i,r in zip(items_left, recipe)]
    cart.append(recipe)
    return items_left

def remove_from_cart(cart, recipe, items_left):
    recipe = list(map(int, recipe))
    items_left = [i+r for i,r in zip(items_left, recipe)]
    cart.remove(recipe)
    return items_left

def add_to_full_cart(full_cart, recipe, recipe_score):
    #full_cart = [scores : list, cart : list, items_left : list]
    scores = full_cart[0]
    cart = full_cart[1]
    items_left = add_to_cart(cart, recipe, full_cart[2])
    scores.append(recipe_score)
    full_cart[2] = items_left

def remove_from_full_cart(full_cart, recipe):
    #full_cart = [scores : list, cart : list, items_left : list]
    recipe = list(map(int, recipe))
    scores = full_cart[0]
    cart = full_cart[1]
    last_index = len(cart) - 1 - cart[::-1].index(recipe)
    del cart[last_index]
    del scores[last_index]
    full_cart[2] = [i+r for i,r in zip(full_cart[2], recipe)]
    
def print_full_cart(full_cart):
    unique_c = []
    items_left = full_cart[2]
    cart = full_cart[1]
    score = full_cart[0]
    for c,s in zip(cart,score):
        if c in unique_c: continue
        unique_c.append(c)
        print('Recipe = %s, instances = %d, score = %.2f' % (c, cart.count(c), s))
    print('Items left = %s' % items_left)
    print('Cart length = %d, cart score = %.2f' % (len(cart),sum(score)))

def recipes_to_cart_list(recipe_list, gift_ids, random_list=False):
    _ids = unshared_copy(gift_ids)
    if random_list: shuffle(_ids)
    return [recipe_list_to_ids(r, _ids) for r in recipe_list]

def max_n_of_packs(recipe, items_left, packs_left):
    return min([packs_left] + [(n_left//n_recipe) if n_recipe>0 else np.inf for n_left, n_recipe in zip(items_left, recipe)])

def print_recipe(recipe, gift_types):
    for r,t in zip(recipe,gift_types):
        print('%s: %d' % (t,r))

def print_cart_extract(cart, score):
    unique_c = []
    for c,s in zip(cart,score):
        if c in unique_c: continue
        unique_c.append(c)
        print('Recipe = %s, instances = %d, score = %.2f' % (c, cart.count(c), s))
        
def get_recipe(dataframe, row):
    return list(dataframe.iloc[row,:9].astype(int))

def get_full_recipe(dataframe, row):
    return [dataframe.iloc[row,9], list(dataframe.iloc[row,:9].astype(int))]

def choiceful_gnome(dataframe, gift_numbers, N_packs=1000):
    dataframe = dataframe.copy().reset_index(drop=True)
    packs_left = N_packs
    items_left = gift_numbers[:]
    cart_recipes = []
    cart_scores = []
    while packs_left != 0:
        max_packs = dataframe.apply(lambda x: min([packs_left] + [(n_left//n_recipe) if n_recipe>0 else np.inf for n_left, n_recipe in zip(items_left, x[:9])]),axis=1)
        scores = dataframe['score']
        total_score = max_packs * scores
        max_index = total_score.idxmax()
        n = int(max_packs[max_index])
        if n ==0:
            print('!!!Incomplete cart!!!')
            print('Cart length = %.2d\nCart average = %.2f' % (len(cart_recipes),sum(cart_scores)/len(cart_recipes)))
            break
        max_score = scores[max_index]
        max_recipe = dataframe.iloc[max_index,:9]
        cart_recipes += [list(map(int,max_recipe))] * n
        cart_scores += [max_score] * n
        packs_left -= n
        items_left -= max_recipe * n
        if min(items_left)<0:
            print(packs_left)
            print(max_packs[max_index])
            #print(cart_recipes)
            print('packs_left = %d\n' % packs_left)
            print('n = %d' % n)
            print('max_score = %d' % max_score)
            print('max_index = %d' % max_index)
            print('max_recipe = %s\n' % list(max_recipe))
            print('items_left =\n%s' % list(items_left))
            raise Exception('Something went wrong')
    return [cart_scores, cart_recipes, list(items_left.astype(int))]
        
        
        
        

In [120]:
a = choiceful_gnome(df[(df.max_packs < 500)&(df.n_items>=3)], gift_numbers)

In [532]:
df[(df.ball==7)&(df.book==2)&(df.doll==1)&(df.gloves==2)&(df['blocks']==1)]

,horse,ball,bike,train,coal,book,doll,blocks,gloves,score,n_items,max_packs
3000,0,7,0,0,0,2,1,1,2,37.041179,13,100


In [250]:
b = recipes_to_cart_list(a[1],gift_ids)

In [246]:
b

['horse_999 horse_998 ball_1099 book_1199 book_1198 book_1197 doll_999 doll_998 blocks_999',
 'horse_997 horse_996 ball_1098 book_1196 book_1195 book_1194 doll_997 doll_996 blocks_998',
 'horse_995 horse_994 ball_1097 book_1193 book_1192 book_1191 doll_995 doll_994 blocks_997',
 'horse_993 horse_992 ball_1096 book_1190 book_1189 book_1188 doll_993 doll_992 blocks_996',
 'horse_991 horse_990 ball_1095 book_1187 book_1186 book_1185 doll_991 doll_990 blocks_995',
 'horse_989 horse_988 ball_1094 book_1184 book_1183 book_1182 doll_989 doll_988 blocks_994',
 'horse_987 horse_986 ball_1093 book_1181 book_1180 book_1179 doll_987 doll_986 blocks_993',
 'horse_985 horse_984 ball_1092 book_1178 book_1177 book_1176 doll_985 doll_984 blocks_992',
 'horse_983 horse_982 ball_1091 book_1175 book_1174 book_1173 doll_983 doll_982 blocks_991',
 'horse_981 horse_980 ball_1090 book_1172 book_1171 book_1170 doll_981 doll_980 blocks_990',
 'horse_979 horse_978 ball_1089 book_1169 book_1168 book_1167 doll_979

In [271]:
for i in reversed(range(3,28)):
    print('-----\ni=%d' % i)
    a = choiceful_gnome(df[(df.gloves < i)&(df.n_items>=3)], gift_numbers)
    print('Cart score = %.2f' % sum(a[0]))

-----
i=27
Cart score = 35030.03
-----
i=26
Cart score = 35030.03
-----
i=25
Cart score = 35030.03
-----
i=24
Cart score = 35030.03
-----
i=23
Cart score = 35030.03
-----
i=22
Cart score = 35030.03
-----
i=21
Cart score = 35030.03
-----
i=20
Cart score = 35030.03
-----
i=19
Cart score = 35030.03
-----
i=18
Cart score = 35030.03
-----
i=17
Cart score = 35030.03
-----
i=16
Cart score = 35030.03
-----
i=15
Cart score = 35030.03
-----
i=14
Cart score = 35030.03
-----
i=13
Cart score = 35030.03
-----
i=12
Cart score = 35030.03
-----
i=11
Cart score = 35030.03
-----
i=10
Cart score = 35030.03
-----
i=9
Cart score = 35030.03
-----
i=8
Cart score = 35030.03
-----
i=7
Cart score = 35030.03
-----
i=6
Cart score = 35030.03
-----
i=5
Cart score = 35030.03
-----
i=4
Cart score = 35030.03
-----
i=3
Cart score = 35030.03


In [103]:
a = choiceful_gnome(df[(df.max_packs < 250)&(df.n_items>=3)],gift_types, gift_numbers)

!!!Incomplete cart!!!
Cart length = 219
Cart average = 37.66


In [203]:
c = [None] * 10
' '.join(c)

TypeError: sequence item 0: expected str instance, NoneType found

In [4]:
a=df.apply(lambda x: x[9]*min([(n_left//n_recipe)if n_recipe>0 else np.inf  for n_left, n_recipe in zip(gift_numbers, x[:9])]),axis=1)

In [ ]:
def foo(a,b):
    if b == 0:
        return np.inf
    else:
        return a//b
    
def bar(recipe, gift_numbers):
    return min([foo(n_left,n_recipe) for n_left, n_recipe in zip(gift_numbers, recipe)])
    
df.apply(lambda x:bar(x, gift_numbers), axis=1)

In [ ]:
a = df.apply(lambda x: x[9]*min([(n_left//n_recipe)if n_recipe>0 else np.inf  for n_left, n_recipe in zip(gift_numbers, x[:9])]),axis=1)
a.rename('total_score')
b = df['max_packs']
c = pd.concat([a,b],axis = 1)

In [ ]:
c.columns = ['total_score', 'max_packs']
c.sort_values('total_score',ascending=False)

In [ ]:
a = df[gift_types].loc[0][0:9]

In [ ]:
int(np.inf)

35545.788946319837

In [254]:
cg = pd.DataFrame(b, columns=['Gifts'])

In [256]:
cg.to_csv('choiceful1.csv', index=False)

In [257]:
choiceful1_score = 35580.20701
expected_cf1_score = 35545.788946319837

In [265]:
sum(a[0])

35545.788946319837

In [258]:
a[2]

horse       0.0
ball       11.0
bike      434.0
train       1.0
coal      166.0
book        0.0
doll        2.0
blocks      0.0
gloves    134.0
Name: 2987, dtype: float64

In [264]:
df[(df.ball<=11)&(df.train==1)&(df.horse==0)]

,horse,ball,bike,train,coal,book,doll,blocks,gloves,score,n_items,max_packs
1322,0,11,0,1,0,0,1,0,2,37.988951,15,100
1434,0,10,0,1,0,0,2,0,0,37.898440,13,110
1443,0,9,0,1,0,0,0,1,0,37.887593,11,122
1598,0,11,0,1,0,1,1,0,1,37.777489,15,100
1654,0,11,0,1,0,1,1,0,0,37.747329,14,100
1784,0,11,0,1,0,0,1,0,1,37.659485,14,100
1789,0,11,0,1,0,0,0,0,5,37.656805,17,40
1847,0,10,0,1,0,0,1,0,3,37.618459,15,66
1908,0,11,0,1,0,1,0,0,4,37.589615,17,50
1944,0,9,0,1,0,0,2,0,1,37.561310,13,122


In [276]:
a[0]

[37.045036838586881,
 37.045036838586881,
 37.045036838586881,
 37.045036838586881,
 37.045036838586881,
 37.045036838586881,
 37.045036838586881,
 37.045036838586881,
 37.045036838586881,
 37.045036838586881,
 37.045036838586881,
 37.045036838586881,
 37.045036838586881,
 37.045036838586881,
 37.045036838586881,
 37.045036838586881,
 37.045036838586881,
 37.045036838586881,
 37.045036838586881,
 37.045036838586881,
 37.045036838586881,
 37.045036838586881,
 37.045036838586881,
 37.045036838586881,
 37.045036838586881,
 37.045036838586881,
 37.045036838586881,
 37.045036838586881,
 37.045036838586881,
 37.045036838586881,
 37.045036838586881,
 37.045036838586881,
 37.045036838586881,
 37.045036838586881,
 37.045036838586881,
 37.045036838586881,
 37.045036838586881,
 37.045036838586881,
 37.045036838586881,
 37.045036838586881,
 37.045036838586881,
 37.045036838586881,
 37.045036838586881,
 37.045036838586881,
 37.045036838586881,
 37.045036838586881,
 37.045036838586881,
 37.045036838

In [7]:
df_all = pd.read_csv('all_possible_combs.csv')

In [278]:
len(df_all)

1131212

In [8]:
df_all = df_all[(df_all.score>20)&(df_all.n_items >=3)]
df_all = df_all.sort_values('score', ascending=False)
df_all = df_all.reset_index()
del df_all['index']

In [294]:
len(df_all)

32316

In [119]:
for i in reversed([k*50 for k in range(1,21)]):
    print('-----\ni=%d' % i)
    a = choiceful_gnome(df_all[(df_all.max_packs <= i)], gift_numbers)
    print('Cart score = %.2f' % sum(a[0]))

-----
i=1000
Cart score = 35030.03
-----
i=950
Cart score = 34064.78
-----
i=900
Cart score = 34064.78
-----
i=850
Cart score = 34064.78
-----
i=800
Cart score = 34064.78
-----
i=750
Cart score = 34064.78
-----
i=700
Cart score = 34064.78
-----
i=650
Cart score = 34064.78
-----
i=600
Cart score = 34064.78
-----
i=550
Cart score = 34042.46
-----
i=500
Cart score = 34595.70
-----
i=450
Cart score = 35545.79
-----
i=400
Cart score = 35545.79
-----
i=350
Cart score = 34874.42
-----
i=300
Cart score = 34511.64
-----
i=250
Cart score = 34445.11
-----
i=200
!!!Incomplete cart!!!
Cart length = 782
Cart average = 33.87
Cart score = 26485.15
-----
i=150
!!!Incomplete cart!!!
Cart length = 429
Cart average = 37.21
Cart score = 15964.76
-----
i=100
!!!Incomplete cart!!!
Cart length = 330
Cart average = 36.04
Cart score = 11891.60
-----
i=50
!!!Incomplete cart!!!
Cart length = 137
Cart average = 35.25
Cart score = 4829.33


In [349]:
df_all.iloc[1,:]

index        1.068655e+06
horse        0.000000e+00
ball         2.200000e+01
bike         0.000000e+00
train        0.000000e+00
coal         0.000000e+00
book         0.000000e+00
doll         0.000000e+00
blocks       0.000000e+00
gloves       1.000000e+00
score        4.487567e+01
n_items      2.300000e+01
max_packs    5.000000e+01
Name: 1, dtype: float64

In [352]:
df_all

,horse,ball,bike,train,coal,book,doll,blocks,gloves,score,n_items,max_packs
0,0,23,0,0,0,0,0,0,0,45.863013,23,47
1,0,22,0,0,0,0,0,0,1,44.875672,23,50
2,0,21,0,0,0,0,0,0,2,44.002327,23,52
3,0,22,0,0,0,0,0,0,0,43.999495,22,50
4,0,21,0,0,0,1,0,0,0,43.659044,22,52
5,0,24,0,0,0,0,0,0,0,43.602084,24,45
6,0,22,0,0,0,1,0,0,0,43.400335,23,50
7,0,21,0,0,0,0,0,0,1,43.380163,22,52
8,0,20,0,0,0,0,0,0,3,43.260245,23,55
9,0,20,0,0,0,1,0,0,1,42.943586,22,55


In [491]:
a = choiceful_gnome(df[(df.max_packs < 500)&(df.n_items>=3)], gift_numbers)
sum(a[0])
fc2 = deepcopy(a)

In [288]:
print_full_cart_extract(full_cart)

Recipe = [2, 1, 0, 0, 0, 3, 2, 1, 0], instances = 400, score = 37.05
Recipe = [0, 2, 0, 3, 0, 0, 0, 0, 0], instances = 333, score = 33.15
Recipe = [1, 0, 0, 0, 0, 0, 0, 3, 0], instances = 200, score = 37.87
Recipe = [0, 0, 1, 0, 0, 0, 3, 0, 1], instances = 66, score = 31.35
Recipe = [0, 23, 0, 0, 0, 0, 0, 0, 0], instances = 1, score = 45.86
Items left = [0, 11, 434, 1, 166, 0, 2, 0, 134]
Cart length = 1000


In [295]:
recipe = full_cart[1][-2]
score = full_cart[0][-2]

In [301]:
print_recipe(recipe, gift_types)

horse: 0
ball: 0
bike: 1
train: 0
coal: 0
book: 0
doll: 3
blocks: 0
gloves: 1


In [403]:
print_full_cart(mock_full_cart)

Recipe = [2, 1, 0, 0, 0, 3, 2, 1, 0], instances = 400, score = 37.05
Recipe = [0, 2, 0, 3, 0, 0, 0, 0, 0], instances = 333, score = 33.15
Recipe = [1, 0, 0, 0, 0, 0, 0, 3, 0], instances = 200, score = 37.87
Recipe = [0, 0, 1, 0, 0, 0, 3, 0, 1], instances = 66, score = 31.35
Recipe = [0, 23, 0, 0, 0, 0, 0, 0, 0], instances = 1, score = 45.86
Items left = [0, 11, 434, 1, 166, 0, 2, 0, 134]
Cart length = 1000


In [404]:
remove_from_full_cart(mock_full_cart,mock_full_cart[1][-1]) 
print_full_cart(mock_full_cart)

Recipe = [2, 1, 0, 0, 0, 3, 2, 1, 0], instances = 400, score = 37.05
Recipe = [0, 2, 0, 3, 0, 0, 0, 0, 0], instances = 333, score = 33.15
Recipe = [1, 0, 0, 0, 0, 0, 0, 3, 0], instances = 200, score = 37.87
Recipe = [0, 0, 1, 0, 0, 0, 3, 0, 1], instances = 66, score = 31.35
Items left = [0, 34, 434, 1, 166, 0, 2, 0, 134]
Cart length = 999


In [410]:
df[(df.ball<7)&(df.gloves>0)&(df.score>36.5)&(df.horse==0)&(df.book==0)&(df['blocks']==0)]

,horse,ball,bike,train,coal,book,doll,blocks,gloves,score,n_items,max_packs
1058,0,6,0,0,0,0,5,0,2,38.228347,13,100
1301,0,6,0,0,0,0,4,0,6,38.002859,16,33
1446,0,5,0,0,0,0,5,0,4,37.885631,14,50
1562,0,6,0,0,0,0,4,0,5,37.801023,15,40
1573,0,5,0,0,0,0,5,0,3,37.796930,13,66
1604,0,4,0,0,0,0,6,0,1,37.773523,11,166
1674,0,6,0,0,0,0,3,0,9,37.730369,18,22
1718,0,6,0,0,0,0,5,0,1,37.700157,12,183
1765,0,5,0,0,0,0,4,0,7,37.668262,16,28
1911,0,4,0,0,0,0,5,0,5,37.587007,14,40


In [417]:
fc = deepcopy(full_cart)
print_full_cart(fc)

Recipe = [2, 1, 0, 0, 0, 3, 2, 1, 0], instances = 400, score = 37.05
Recipe = [0, 2, 0, 3, 0, 0, 0, 0, 0], instances = 333, score = 33.15
Recipe = [1, 0, 0, 0, 0, 0, 0, 3, 0], instances = 200, score = 37.87
Recipe = [0, 0, 1, 0, 0, 0, 3, 0, 1], instances = 66, score = 31.35
Recipe = [0, 23, 0, 0, 0, 0, 0, 0, 0], instances = 1, score = 45.86
Items left = [0, 11, 434, 1, 166, 0, 2, 0, 134]
Cart length = 1000


In [419]:
[new_score, new_recipe] = get_full_recipe(df,1301)
old_recipe = fc[1][-2]
old_recipe

[0, 0, 1, 0, 0, 0, 3, 0, 1]

In [424]:
remove_from_full_cart(fc, fc[1][-1])
print_full_cart(fc)

Recipe = [2, 1, 0, 0, 0, 3, 2, 1, 0], instances = 400, score = 37.05
Recipe = [0, 2, 0, 3, 0, 0, 0, 0, 0], instances = 333, score = 33.15
Recipe = [1, 0, 0, 0, 0, 0, 0, 3, 0], instances = 200, score = 37.87
Recipe = [0, 0, 1, 0, 0, 0, 3, 0, 1], instances = 57, score = 31.35
Items left = [0, 34, 443, 1, 166, 0, 29, 0, 143]
Cart length = 990


In [427]:
new_recipe

[0, 6, 0, 0, 0, 0, 4, 0, 6]

In [443]:
add_to_full_cart(fc, new_recipe, new_score)

[0, 4, 443, 1, 166, 0, 9, 0, 113]
[0, 6, 0, 0, 0, 0, 4, 0, 6]
[0, -2, 443, 1, 166, 0, 5, 0, 107]


Exception: Negative number of items left.

In [447]:
print_full_cart(fc)

Recipe = [2, 1, 0, 0, 0, 3, 2, 1, 0], instances = 400, score = 37.05
Recipe = [0, 2, 0, 3, 0, 0, 0, 0, 0], instances = 333, score = 33.15
Recipe = [1, 0, 0, 0, 0, 0, 0, 3, 0], instances = 200, score = 37.87
Recipe = [0, 0, 1, 0, 0, 0, 3, 0, 1], instances = 57, score = 31.35
Recipe = [0, 6, 0, 0, 0, 0, 4, 0, 6], instances = 5, score = 38.00
Items left = [0, 4, 443, 1, 166, 0, 9, 0, 113]
Cart length = 995, cart score = 35407.78


In [458]:
df[(df['score']>35)&(df['horse']==0)&(df['blocks']==0)&(df['ball']==0)&(df['doll']==0)&(df['train']<=1)]

,horse,ball,bike,train,coal,book,doll,blocks,gloves,score,n_items,max_packs
8469,0,0,0,0,0,0,0,0,28,35.301604,28,7
8501,0,0,0,0,0,0,0,0,27,35.290801,27,7
8684,0,0,0,0,0,1,0,0,26,35.239918,27,7
9152,0,0,0,0,0,2,0,0,25,35.095696,27,8
9263,0,0,0,0,0,2,0,0,24,35.062451,26,8
9403,0,0,0,0,0,3,0,0,23,35.022827,26,8
9443,0,0,0,0,0,1,0,0,25,35.013911,26,8


In [461]:
index = 8469
[new_score, new_recipe] = get_full_recipe(df,index)
add_to_full_cart(fc, new_recipe, new_score)
print_full_cart(fc)

Recipe = [2, 1, 0, 0, 0, 3, 2, 1, 0], instances = 400, score = 37.05
Recipe = [0, 2, 0, 3, 0, 0, 0, 0, 0], instances = 333, score = 33.15
Recipe = [1, 0, 0, 0, 0, 0, 0, 3, 0], instances = 200, score = 37.87
Recipe = [0, 0, 1, 0, 0, 0, 3, 0, 1], instances = 57, score = 31.35
Recipe = [0, 6, 0, 0, 0, 0, 4, 0, 6], instances = 5, score = 38.00
Recipe = [0, 4, 0, 0, 0, 0, 4, 0, 8], instances = 1, score = 37.30
Recipe = [0, 0, 0, 0, 0, 0, 5, 0, 10], instances = 1, score = 36.36
Recipe = [0, 0, 0, 0, 0, 0, 0, 0, 28], instances = 3, score = 35.30
Items left = [0, 0, 443, 1, 166, 0, 0, 0, 11]
Cart length = 1000, cart score = 35587.35


In [464]:
tweaked_cg = pd.DataFrame(recipes_to_cart_list(fc[1], gift_ids), columns=['Gifts'])

In [467]:
tweaked_cg.to_csv('tweaked_cg.csv', index=False)

In [468]:
tweaked_cg_score = 35537.60297

In [492]:
print_full_cart(fc2)

Recipe = [2, 1, 0, 0, 0, 3, 2, 1, 0], instances = 400, score = 37.05
Recipe = [0, 2, 0, 3, 0, 0, 0, 0, 0], instances = 333, score = 33.15
Recipe = [1, 0, 0, 0, 0, 0, 0, 3, 0], instances = 200, score = 37.87
Recipe = [0, 0, 1, 0, 0, 0, 3, 0, 1], instances = 66, score = 31.35
Recipe = [0, 23, 0, 0, 0, 0, 0, 0, 0], instances = 1, score = 45.86
Items left = [0, 11, 434, 1, 166, 0, 2, 0, 134]
Cart length = 1000, cart score = 35545.79


In [493]:
sum(fc2[0])

35545.788946319837

In [482]:
df[(df['score']>31.35)&(df['horse']==0)&(df['blocks']==0)&(df['ball']<=6)&(df['doll']<=3)&(df['train']<=1)&(df['book']==0)]

,horse,ball,bike,train,coal,book,doll,blocks,gloves,score,n_items,max_packs
1674,0,6,0,0,0,0,3,0,9,37.730369,18,22
2070,0,6,0,0,0,0,2,0,13,37.486438,21,15
2292,0,5,0,0,0,0,3,0,11,37.367735,19,18
2334,0,6,0,0,0,0,2,0,12,37.349158,20,16
2361,0,5,0,0,0,0,3,0,10,37.334532,18,20
2460,0,6,0,0,0,0,3,0,8,37.289410,17,25
2525,0,6,0,0,0,0,1,0,16,37.252370,23,12
2684,0,5,0,0,0,0,2,0,14,37.185308,21,14
2861,0,4,0,0,0,0,3,0,12,37.101026,19,16
3104,0,6,0,0,0,0,0,0,20,36.999371,26,10


In [495]:
remove_from_full_cart(fc2, fc2[1][-2])
print_full_cart(fc2)

Recipe = [2, 1, 0, 0, 0, 3, 2, 1, 0], instances = 400, score = 37.05
Recipe = [0, 2, 0, 3, 0, 0, 0, 0, 0], instances = 333, score = 33.15
Recipe = [1, 0, 0, 0, 0, 0, 0, 3, 0], instances = 200, score = 37.87
Recipe = [0, 0, 1, 0, 0, 0, 3, 0, 1], instances = 64, score = 31.35
Recipe = [0, 23, 0, 0, 0, 0, 0, 0, 0], instances = 1, score = 45.86
Items left = [0, 11, 436, 1, 166, 0, 8, 0, 136]
Cart length = 998, cart score = 35483.09


In [497]:
index = 1446
index = 1674
[new_score, new_recipe] = get_full_recipe(df,index)
add_to_full_cart(fc2, new_recipe, new_score)
print_full_cart(fc2)

Recipe = [2, 1, 0, 0, 0, 3, 2, 1, 0], instances = 400, score = 37.05
Recipe = [0, 2, 0, 3, 0, 0, 0, 0, 0], instances = 333, score = 33.15
Recipe = [1, 0, 0, 0, 0, 0, 0, 3, 0], instances = 200, score = 37.87
Recipe = [0, 0, 1, 0, 0, 0, 3, 0, 1], instances = 64, score = 31.35
Recipe = [0, 23, 0, 0, 0, 0, 0, 0, 0], instances = 1, score = 45.86
Recipe = [0, 5, 0, 0, 0, 0, 5, 0, 4], instances = 1, score = 37.89
Recipe = [0, 6, 0, 0, 0, 0, 3, 0, 9], instances = 1, score = 37.73
Items left = [0, 0, 436, 1, 166, 0, 0, 0, 123]
Cart length = 1000, cart score = 35558.70


In [499]:
tweaked_cg2 = pd.DataFrame(recipes_to_cart_list(fc2[1], gift_ids), columns=['Gifts'])
tweaked_cg2 

,Gifts
0,horse_999 horse_998 ball_1099 book_1199 book_1...
1,horse_997 horse_996 ball_1098 book_1196 book_1...
2,horse_995 horse_994 ball_1097 book_1193 book_1...
3,horse_993 horse_992 ball_1096 book_1190 book_1...
4,horse_991 horse_990 ball_1095 book_1187 book_1...
5,horse_989 horse_988 ball_1094 book_1184 book_1...
6,horse_987 horse_986 ball_1093 book_1181 book_1...
7,horse_985 horse_984 ball_1092 book_1178 book_1...
8,horse_983 horse_982 ball_1091 book_1175 book_1...
9,horse_981 horse_980 ball_1090 book_1172 book_1...


In [500]:
tweaked_cg2.to_csv('tweaked_cg2.csv', index=False)

In [501]:
tweaked_cg2_score = 35561.57817